In [1]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
#inital reading of the data
depression=pd.read_csv('master.csv')
depression.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [3]:
#transforming gdp_for_year($) from string into an int for computational/analytic purposes
d=depression.drop(['country-year','HDI for year'],axis=1)
v=(d.loc[:,' gdp_for_year ($) '])
#change gdp from str to float, so mean is possible
v=v.str.replace(',','')
v=pd.to_numeric(v)
d['gdp_for_year ($)']=v
d=d.drop([' gdp_for_year ($) '],axis=1)
d

,country,year,sex,age,suicides_no,population,suicides/100k pop,gdp_per_capita ($),generation,gdp_for_year ($)
0,Albania,1987,male,15-24 years,21,312900,6.71,796,Generation X,2156624900
1,Albania,1987,male,35-54 years,16,308000,5.19,796,Silent,2156624900
2,Albania,1987,female,15-24 years,14,289700,4.83,796,Generation X,2156624900
3,Albania,1987,male,75+ years,1,21800,4.59,796,G.I. Generation,2156624900
4,Albania,1987,male,25-34 years,9,274300,3.28,796,Boomers,2156624900
...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,2309,Generation X,63067077179
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,2309,Silent,63067077179
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,2309,Generation Z,63067077179
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,2309,Generation Z,63067077179


In [4]:
#EDA, getting a feel for the data
d.describe()

,year,suicides_no,population,suicides/100k pop,gdp_per_capita ($),gdp_for_year ($)
count,27820.000000,27820.000000,2.782000e+04,27820.000000,27820.000000,2.782000e+04
mean,2001.258375,242.574407,1.844794e+06,12.816097,16866.464414,4.455810e+11
std,8.469055,902.047917,3.911779e+06,18.961511,18887.576472,1.453610e+12
min,1985.000000,0.000000,2.780000e+02,0.000000,251.000000,4.691962e+07
25%,1995.000000,3.000000,9.749850e+04,0.920000,3447.000000,8.985353e+09
50%,2002.000000,25.000000,4.301500e+05,5.990000,9372.000000,4.811469e+10
75%,2008.000000,131.000000,1.486143e+06,16.620000,24874.000000,2.602024e+11
max,2016.000000,22338.000000,4.380521e+07,224.970000,126352.000000,1.812071e+13


In [5]:
#removing outliers & cleaning up data
depression.describe()
#the outlier in the suicides/100k pop is super extreme compared to the rest
#depression[depression['suicides/100k pop']==224.97]
d=d[(d['suicides/100k pop']<100) & (d['suicides_no']>=1)]
#dd=depression[depression['suicides_no']>=1]
d.describe()

,year,suicides_no,population,suicides/100k pop,gdp_per_capita ($),gdp_for_year ($)
count,23353.000000,23353.000000,2.335300e+04,23353.000000,23353.000000,2.335300e+04
mean,2001.317390,282.258211,2.172742e+06,14.293268,17288.815570,5.248001e+11
std,8.463061,935.354871,4.178816e+06,17.215690,19152.428349,1.573480e+12
min,1985.000000,1.000000,1.003000e+03,0.020000,251.000000,4.691962e+07
25%,1995.000000,8.000000,2.423000e+05,2.740000,3299.000000,1.599989e+10
50%,2002.000000,42.000000,5.934090e+05,8.140000,9773.000000,7.714800e+10
75%,2009.000000,172.000000,2.142391e+06,19.190000,25848.000000,3.253583e+11
max,2016.000000,21262.000000,4.380521e+07,99.990000,126352.000000,1.812071e+13


In [6]:
#switching to numerical values, taking the mean to get a gen approx of values. 
#this is intuition for solving PCA
d3=d.groupby(['country']).mean()
d3=d3.drop(['year'],axis=1)
d3
#should we use PCA? not many columns in the first place

,suicides_no,population,suicides/100k pop,gdp_per_capita ($),gdp_for_year ($)
country,,,,,
Albania,9.563107,2.368748e+05,4.489126,1650.140777,4.648709e+09
Antigua and Barbuda,1.100000,7.371200e+03,17.914000,11328.300000,8.826761e+08
Argentina,221.018817,2.784907e+06,10.469328,7914.096774,2.742565e+11
Armenia,7.559524,2.572360e+05,3.873849,1910.027778,5.485033e+09
Aruba,1.959184,1.011122e+04,23.204490,24730.020408,2.264434e+09
...,...,...,...,...,...
United Arab Emirates,13.234043,7.016227e+05,2.018936,41916.744681,2.542419e+11
United Kingdom,370.745257,4.683143e+06,7.563469,31983.937669,1.820855e+12
United States,2779.604839,2.165061e+07,13.819812,39269.612903,1.051071e+13


In [7]:
#PCA!
d3_centered=d3-np.mean(d3,axis=0)
d3_stand=d3_centered/np.std(d3,axis=0)
d3_stand

,suicides_no,population,suicides/100k pop,gdp_per_capita ($),gdp_for_year ($)
country,,,,,
Albania,-0.377446,-0.435386,-1.146183,-0.888320,-0.300743
Antigua and Barbuda,-0.394010,-0.508441,0.418067,-0.301669,-0.303914
Argentina,0.036399,0.375696,-0.449378,-0.508624,-0.073678
Armenia,-0.381367,-0.428905,-1.217875,-0.872567,-0.300038
Aruba,-0.392328,-0.507569,1.034508,0.510690,-0.302751
...,...,...,...,...,...
United Arab Emirates,-0.370262,-0.287449,-1.434007,1.552480,-0.090534
United Kingdom,0.329432,0.979938,-0.787965,0.950393,1.228872
United States,5.043867,6.380978,-0.058983,1.392021,8.547498


In [8]:
u, s, vt =np.linalg.svd(d3_stand,full_matrices=False)
##python has a function that solves the SVD of our matrix d3_stand
##here are the singular values
s

array([16.10879316, 10.23932038,  9.88845866,  4.74512248,  3.21951842])

In [9]:
#PCA Matrix 
P=d3_stand@vt.T
P

,0,1,2,3,4
country,,,,,
Albania,-0.849755,1.244727,0.390862,-0.014654,-0.317657
Antigua and Barbuda,-0.704954,-0.382020,0.289923,0.205593,-0.051777
Argentina,0.087060,0.631561,0.398512,-0.101662,0.181507
Armenia,-0.850371,1.307699,0.355604,-0.029500,-0.321689
Aruba,-0.535879,-1.192561,-0.279076,0.179634,0.121236
...,...,...,...,...,...
United Arab Emirates,-0.287634,0.825172,-1.941042,-0.385704,-0.120143
United Kingdom,1.539541,0.722797,-1.059431,0.275531,0.110158
United States,11.580778,1.206860,-0.479195,2.273749,-0.433490


In [10]:
total_var=sum(np.var(d3_stand,axis=0))
total_var

5.0

In [11]:
total_var2=sum(np.var(P,axis=0))
total_var2

5.000000000000002

As we can see, the principal component matrix explains approximately the same amount of variance as the standardized matrix. 

In [12]:
#percent variance explained by principal components
var_perc=[np.var(P, axis=0)[i]/total_var for i in range(5)]
var_perc

[0.5242287215595769,
 0.21180541804424075,
 0.197538615587802,
 0.04548724712831918,
 0.02093999768006149]

In [13]:
var_perc2=[np.var(d3_stand, axis=0)[i]/total_var for i in range(5)]
var_perc2

[0.19999999999999998,
 0.20000000000000004,
 0.2000000000000001,
 0.1999999999999999,
 0.20000000000000004]

In [14]:
#Scree Plot
df=pd.DataFrame({
    'PC #': [1,2,3,4,5], 
    'Fraction of Variance Explained' : var_perc})

alt.Chart(df).mark_bar(size=50).encode(
        x='PC #',
        y='Fraction of Variance Explained'
).properties(title='Scree Plot of PC Matrix')
#first PC describes over half the data

alt.Chart(...)

In [15]:
df2=pd.DataFrame({
    'Variable': d3.columns, 
    'Fraction of Variance Explained' : var_perc2})

alt.Chart(df2).mark_bar(size=10).encode(
        x='Variable',
        y='Fraction of Variance Explained'
).properties(title='Scree Plot of Original Matrix')
#compare this with the percent variance explained by original columns

alt.Chart(...)

In [16]:
#plotting our PCA, this doesn't intuitively make sense (negative values, clustered more towards PC2)
#so, we discard PCA for exploratory data analysis
P.columns=['PC1','PC2','PC3','PC4','PC5']
alt.Chart(P).mark_point().encode(
        x='PC1',
        y='PC2').properties(title='Principal Component Graph')

alt.Chart(...)

In [17]:
#trying to identify relationship between age group + suicide
d2=d.groupby(['age','year']).mean().reset_index()
d2

,age,year,suicides_no,population,suicides/100k pop,gdp_per_capita ($),gdp_for_year ($)
0,15-24 years,1985,229.102564,2.516080e+06,10.375000,6144.346154,2.365441e+11
1,15-24 years,1986,223.000000,2.449219e+06,9.661728,7511.320988,2.724068e+11
2,15-24 years,1987,180.571429,2.285950e+06,8.886703,9091.428571,2.850006e+11
3,15-24 years,1988,172.325843,2.197926e+06,9.635843,9915.426966,3.284417e+11
4,15-24 years,1989,196.168421,2.308689e+06,10.028316,10030.315789,3.359798e+11
...,...,...,...,...,...,...,...
186,75+ years,2012,179.364341,9.126579e+05,21.777364,27646.248062,8.554977e+11
187,75+ years,2013,179.390625,9.234966e+05,23.083281,29587.031250,8.729082e+11
188,75+ years,2014,182.884615,9.366415e+05,23.751000,28291.130769,8.421482e+11
189,75+ years,2015,211.504854,1.119346e+06,23.875825,27094.932039,9.206243e+11


In [18]:
#EDA
#box plot exploring variance of suicide rates by age group
alt.Chart(d2).mark_boxplot().encode(
    x='age',y=alt.Y('suicides/100k pop',title='average suicides/100k pop')).properties(title='Boxplot of Avg Suicide Rate vs Age Group')

alt.Chart(...)

In [19]:
#EDA
#plotting suicide rates of the sexes to try to identify a trend between suicide rates + sex
d4=d.groupby(['year','sex']).sum().reset_index(level=['year','sex'])
d4['suicides/100k pop']=100000*d4['suicides_no']/d4['population']
alt.Chart(d4).mark_line().encode(
        x='year:N',
        y='suicides/100k pop',
        color='sex').properties(title='Suicide Rate of Respective Population')


alt.Chart(...)

In [20]:
#EDA
#trying to generalize by country? not so useful
d5=d.groupby(['country','year']).sum().reset_index(level=['country','year'])
d5['suicides/100k pop']=100000*d5['suicides_no']/d5['population']
d5=d5.groupby(['country']).mean().reset_index(level='country')
alt.Chart(d5).mark_bar().encode(
        x=alt.X('suicides/100k pop',title='Average suicides/100k pop across 32 years'),
        y=alt.Y('country',sort='-x')).properties(title='Average Suicide Rate of Countries')

alt.Chart(...)

In [21]:
#EDA: finding a relationship between generation + suicide
d6=d.groupby(['generation','year']).sum().reset_index(level=['generation','year'])
d6['suicides/100k pop']=100000*d6['suicides_no']/d6['population']
d6=d6.groupby(['generation']).mean().reset_index(level='generation')
alt.Chart(d6).mark_bar().encode(
        x=alt.X('suicides/100k pop',title='average suicides/100k pop over 32 years'),
        y=alt.Y('generation',sort='-x')).properties(title='Generation vs Avg Suicide Rate Barplot')
# we can see that this plot and plot below are very similar

alt.Chart(...)

In [22]:
d7=d.groupby(['age','year']).sum().reset_index(level=['age','year'])
d7['suicides/100k pop']=100000*d7['suicides_no']/d7['population']
d7=d7.groupby(['age']).mean().reset_index(level='age')
alt.Chart(d7).mark_bar().encode(
        x=alt.X('suicides/100k pop',title='average suicides/100k pop over 32 years'),
        y=alt.Y('age',sort='-x')).properties(title='Age vs Avg Suicide Rate Barplot')


alt.Chart(...)

In [23]:
#EDA
#this graph ultimately does not provide much intuition
d8=d.groupby(['country','year']).agg({'suicides_no':['sum'],
                              'population':['sum'],
                              'gdp_per_capita ($)':['mean']}).reset_index(level=['country','year'])
d8.columns=['country','year','suicides_no','population','gdp_per_capita ($)']
d8['suicides/100k pop']=100000*d8['suicides_no']/d8['population']
d8=d8.groupby(['country']).mean().reset_index(level='country')

alt.Chart(d8).mark_point().encode(
        x=alt.X('suicides/100k pop',title='average suicides/100k pop over 32 years'),
        y=alt.Y('gdp_per_capita ($)',title='average gdp_per_capita($) over 32 years'),color='country'
).properties(title='Average GDP vs Average Suicide Rate of Countries')

alt.Chart(...)

In [24]:
#EDA
#try a specific year 
d10=d[d['year']==2016]
alt.Chart(d10).mark_point().encode(
        x='suicides/100k pop',
        y='gdp_per_capita ($)').properties(title='2016 Avg GDP vs Avg Suicide Rate')
#interesting to note that all outlines of high suicide rates belong to poor gdp countries

alt.Chart(...)

In [25]:
#EDA
#this graph is visually a little hard to interpret
d9=d.groupby(['year','country']).sum().reset_index(level=['year','country'])
d9['suicides/100k pop']=100000*d9['suicides_no']/d9['population']
alt.Chart(d9).mark_point().encode(
        x='population',
        y='suicides/100k pop').properties(title='Suicide Rate vs Population of Country in Given Year')

alt.Chart(...)

In [26]:
#transformation of the variable sex: making it a binary variable
dd=d.replace({'sex':{'male':0,'female':1}})

In [27]:
def avg_squared_loss(y, y_hat):
    e=y-y_hat
    return (sum(e**2))/len(y)

def add_bias(data):
    data.insert(0,'ones',[float(1) for i in range(len(data))])

In [28]:
#Linear Regression Model 
#training our model and using one hot encoding to predict future trends
from sklearn.linear_model import LinearRegression
X=dd[['population','age','generation','sex','gdp_per_capita ($)']]
add_bias(X)
Y=dd['suicides/100k pop']
from sklearn.feature_extraction import DictVectorizer

#one-hot encoding 


hot = X[['age','generation']].to_dict(orient='records')

encoder = DictVectorizer(sparse=False)
hot_df = pd.DataFrame(
    data = encoder.fit_transform(hot),
    columns = encoder.feature_names_
)


# adjusting the index inconsistency issue
X.reset_index(drop=True, inplace=True)
hot_df.reset_index(drop=True, inplace=True)

# Combine the features together with pd.concat
X = pd.concat([X,hot_df],ignore_index=False, axis=1)
X = X.drop(['generation','age'],axis=1)
X.head()

,ones,population,sex,gdp_per_capita ($),age=15-24 years,age=25-34 years,age=35-54 years,age=5-14 years,age=55-74 years,age=75+ years,generation=Boomers,generation=G.I. Generation,generation=Generation X,generation=Generation Z,generation=Millenials,generation=Silent
0,1.0,312900,0,796,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,308000,0,796,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,289700,1,796,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,21800,0,796,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,274300,0,796,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
model = LinearRegression()
model.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:
Y_hat=model.predict(X)
loss=avg_squared_loss(Y,Y_hat)
loss

188.88685427874285

In [31]:
# Second Linear Regression Model 
X2=dd[['population','age','sex']]
add_bias(X2)
Y=dd['suicides/100k pop']

#one-hot encoding 


hot2 = X2[['age']].to_dict(orient='records')

encoder = DictVectorizer(sparse=False)
hot_df2 = pd.DataFrame(
    data = encoder.fit_transform(hot2),
    columns = encoder.feature_names_
)


# adjusting the index inconsistency issue
X2.reset_index(drop=True, inplace=True)
hot_df2.reset_index(drop=True, inplace=True)

# Combine the features together with pd.concat
X2 = pd.concat([X2,hot_df2],ignore_index=False, axis=1)
X2 = X2.drop(['age'],axis=1)
X2.head()

,ones,population,sex,age=15-24 years,age=25-34 years,age=35-54 years,age=5-14 years,age=55-74 years,age=75+ years
0,1.0,312900,0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,308000,0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,289700,1,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,21800,0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,274300,0,0.0,1.0,0.0,0.0,0.0,0.0


In [32]:
model2 = LinearRegression()
model2.fit(X2,Y)
Y_hat2=model2.predict(X2)
loss2=avg_squared_loss(Y,Y_hat2)
loss2
#compute the two losses to see if adding features give us a more efficient model

189.26179545435585

In [33]:
from sklearn.metrics import r2_score
r2_score(Y,Y_hat2)
#this tells us we could potentially improve our model and train it more 

0.36139441833557717

In [34]:
#Prediction not very accurate
df13=dd
df13['prediction']=Y_hat2
df13=df13[df13['country']=='United States']
alt.Chart(df13).mark_point().encode(
        x='suicides/100k pop',
        y='prediction')

alt.Chart(...)